In [2]:
from bert import tokenization

In [3]:
tokenizer = tokenization.FullTokenizer(
      vocab_file='MASS.wordpiece', do_lower_case=False)

In [4]:
import random

rng = random.Random(12345)

In [5]:
max_seq_length = 300
dupe_factor = 5
short_seq_prob = 0.1
masked_lm_prob = 0.2
max_predictions_per_seq = 20
eos_id = 1

vocab_words = list(tokenizer.vocab.keys())

In [6]:
import collections
import tensorflow as tf
from tqdm import tqdm

maxlen = max_seq_length

def create_int_feature(values):
    feature = tf.train.Feature(
        int64_list = tf.train.Int64List(value = list(values))
    )
    return feature

def to_tfrecord(rows, filename):
    input_encoders, input_decoders, labels = [], [], []
    
    for i in tqdm(range(len(rows))):
        input_encoder, input_decoder, label = rows[0]
        input_encoder = tokenizer.convert_tokens_to_ids(input_encoder)
        input_decoder = tokenizer.convert_tokens_to_ids(input_decoder) + [eos_id]
        label = tokenizer.convert_tokens_to_ids(label) + [eos_id]
        input_encoder = input_encoder + [0] * (maxlen - len(input_encoder))
        input_decoder = input_decoder + [0] * (maxlen - len(input_decoder))
        label = label + [0] * (maxlen - len(label))
        input_encoders.append(input_encoder)
        input_decoders.append(input_decoder)
        labels.append(label)
    
    r = tf.python_io.TFRecordWriter(f'{filename}.tfrecord')
    for i in tqdm(range(len(labels))):
        features = collections.OrderedDict()
        features['input_encoder'] = create_int_feature(input_encoders[i])
        features['input_decoder'] = create_int_feature(input_decoders[i])
        features['y'] = create_int_feature(labels[i])
        tf_example = tf.train.Example(
            features = tf.train.Features(feature = features)
        )
        r.write(tf_example.SerializeToString())
    r.close()
    
def process_NoisyLanguagePairDataset(row):
    max_num_tokens = max_seq_length
    target_seq_length = max_num_tokens
    if rng.random() < short_seq_prob:
        target_seq_length = rng.randint(2, max_num_tokens)
    tokens_l = row[0][:max_num_tokens]
    tokens_r = row[1][:max_num_tokens]
    
    cand_indexes_l = []
    for (i, token) in enumerate(tokens_l):
        if token == '[CLS]' or token == '[SEP]':
            continue
        if (
            len(cand_indexes_l) >= 1
            and token.startswith('##')
        ):
            cand_indexes_l[-1].append(i)
        else:
            cand_indexes_l.append([i])
            
    cand_indexes_r = []
    for (i, token) in enumerate(tokens_r):
        if token == '[CLS]' or token == '[SEP]':
            continue
        if (
            len(cand_indexes_r) >= 1
            and token.startswith('##')
        ):
            cand_indexes_r[-1].append(i)
        else:
            cand_indexes_r.append([i])
            
    rng.shuffle(cand_indexes_l)
    rng.shuffle(cand_indexes_r)

    num_to_predict = min(
        max_predictions_per_seq,
        max(1, int(round(len(tokens_l) * masked_lm_prob))),
    )
    
    masked_lms = []
    output_tokens_l = list(tokens_l)
    covered_indexes = set()
    for index_set in cand_indexes_l:
        if len(masked_lms) >= num_to_predict:
            break
        is_any_index_covered = False
        for index in index_set:
            if index in covered_indexes:
                is_any_index_covered = True
                break
        if is_any_index_covered:
            continue
        for index in index_set:
            covered_indexes.add(index)

            masked_token = None
            # 80% of the time, replace with [MASK]
            if rng.random() < 0.8:
                masked_token = '[MASK]'
            else:
                # 10% of the time, keep original
                if rng.random() < 0.5:
                    masked_token = tokens_l[index]
                # 10% of the time, replace with random word
                else:
                    masked_token = vocab_words[
                        rng.randint(0, len(vocab_words) - 1)
                    ]

            output_tokens_l[index] = masked_token
            masked_lms.append(tokens_l[index])
    
    num_to_predict = min(
        max_predictions_per_seq,
        max(1, int(round(len(tokens_r) * masked_lm_prob))),
    )
    
    output_tokens_r = list(tokens_r)
    label, input_decoder = [], []
    covered_indexes = set()
    for index_set in cand_indexes_r:
        if len(label) >= num_to_predict:
            break
        is_any_index_covered = False
        for index in index_set:
            if index in covered_indexes:
                is_any_index_covered = True
                break
        if is_any_index_covered:
            continue
        for index in index_set:
            covered_indexes.add(index)

            masked_token = None
            # 80% of the time, replace with [MASK]
            if rng.random() < 0.8:
                masked_token = '[MASK]'
            else:
                # 10% of the time, keep original
                if rng.random() < 0.5:
                    masked_token = tokens_r[index]
                # 10% of the time, replace with random word
                else:
                    masked_token = vocab_words[
                        rng.randint(0, len(vocab_words) - 1)
                    ]

            output_tokens_r[index] = masked_token
            label.append(tokens_r[index])
            
    return output_tokens_l, output_tokens_r, tokens_r

In [8]:
dupe_factor = 15

def get_inputs(left, right, filename):
    all_documents = []
    for i in tqdm(range(len(left))):
        line_l = tokenization.convert_to_unicode(left[i])
        line_r = tokenization.convert_to_unicode(right[i])

        line_l = line_l.strip()
        line_r = line_r.strip()
        if line_l and line_r:
            tokens_l = tokenizer.tokenize(line_l)
            tokens_r = tokenizer.tokenize(line_r)
            if len(tokens_l) < max_seq_length and len(tokens_r) < max_seq_length:
                all_documents.append((tokens_l, tokens_r))
                
    results = []
    
    for row in tqdm(all_documents):
        for _ in range(dupe_factor):
            try:
                results.append(process_NoisyLanguagePairDataset(row))
            except:
                pass
            
    to_tfrecord(results, filename)

In [9]:
def chunks_multiple(l, n, filename):
    part = 0
    for i in range(0, len(l), n):
        x, y = list(zip(*l[i : i + n]))
        yield (x, y, f'{filename}-part-{part}')
        part += 1

In [10]:
pairs = ['en-ms.json', 'ms-en.json']

In [11]:
import json
import multi

batch_size = 500000

for pair in pairs:
    with open(pair) as fopen:
        data = json.load(fopen)
    
    part = 0
    for i in range(0, len(data['left']), batch_size):
        index = min(i + batch_size, len(data['left']))
        l = data['left'][i: index]
        r = data['right'][i: index]
        filename = f'{pair}-part-{part}'

        multi.multiprocessing(chunks_multiple(list(zip(l, r)), len(l) // 12, filename), get_inputs)
        part += 1

100%|██████████| 624315/624315 [03:10<00:00, 3277.98it/s]


 30%|██▉       | 186242/624315 [00:22<00:46, 9517.59it/s]]


100%|██████████| 41666/41666 [00:30<00:00, 1371.55it/s]/s]


100%|██████████| 41666/41666 [00:30<00:00, 1357.58it/s]

100%|██████████| 624975/624975 [02:56<00:00, 3545.78it/s]


100%|██████████| 624960/624960 [00:29<00:00, 21284.52it/s]


100%|██████████| 390855/390855 [01:51<00:00, 3517.81it/s]
